In [1]:
from components import *
import numpy as np
import scipy.signal as sci

In [2]:
C, H, W = 2, 5 ,5
h_f, w_f = 2, 2
F = 3
M = 1

In [3]:
def approx_eq(a, b):
    return np.all(np.abs(a - b) < 1e-6)

In [4]:
Hp = H - h_f + 1
Wp = W - w_f + 1

conv = Convolution((C, H, W), (h_f, w_f), F, noActivation())
x = np.random.randn(M, C, H, W)

x_fwd = conv.feedforward(x)
assert x_fwd.shape == (M, F, Hp, Wp)

In [5]:
def test_feedforward(m_test, f_test):
    sum = np.zeros((Hp, Wp))
    for c in range(C):
        img = x[m_test, c] 
        filter = conv.filters[f_test, c] # first filter, channel c
        sum += sci.correlate(img, filter, mode="valid")
    sum += conv.biases[f_test]
    
    assert approx_eq(x_fwd[m_test, f_test], sum)

for m in range(M):
    for f in range(F):
        test_feedforward(m, f)

In [6]:
delta = np.random.randn(M, F, Hp, Wp)
conv.set_optimizer(SGD_optimizer(0, 20).get_optimizer())
prev_delta = conv.backprop(delta)
assert prev_delta.shape == (M, C, H, W)